In [ ]:
import re
import math
from typing import List, Dict, Tuple
import matplotlib.pyplot as plt

# 1) 配置：待处理的 .log 文件及其展示名称（等长一一对应）
LOG_FILES = [
    # 在这里填你的日志文件路径
    "final_test_train_length_512.log",
    "final_test_train_length_1024.log",
    "final_test_train_length_2048.log",
    "final_test_train_length_4096.log",
    "final_test_train_length_8192.log",
    "final_test_train_length_16384.log",
    "final_test_train_length_32768.log",
    "final_test_train_length_65536.log",
    "final_test_train_length_131072.log",
]
LOG_NAMES = [
    # 在这里填画图图例名
    "exp_512",
    "exp_1024",
    "exp_2048",
    "exp_4096",
    "exp_8192",
    "exp_16384",
    "exp_32768",
    "exp_65536",
    "exp_131072",
]

# ========== 正则表达式 (Regular Expression, 正则表达式) ==========
# 匹配 "epoch   i summary:"（空格数量不定，大小写不敏感）
_RE_EPOCH_HDR = re.compile(r"epoch\s+(\d+)\s+summary:", re.IGNORECASE)

# 匹配一行中的若干 "key : value" 对（支持空格/下划线键名、科学计数法数值）
_RE_KV_PAIRS = re.compile(
    r"([A-Za-z0-9_ ]+?)\s*:\s*([+-]?(?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?)"
)

# 匹配每 k 个 epoch 的块状汇总行（train/valid 的 loss/accuracy 四元组）
# 例如：train loss: 2.7947, accuracy: 0.5682, valid loss: 3.8263, accuracy: 0.1567,
_RE_BLOCK = re.compile(
    r"train\s+loss:\s*([+-]?\d+(?:\.\d+)?(?:[eE][+-]?\d+)?)\s*,\s*accuracy:\s*([+-]?\d+(?:\.\d+)?(?:[eE][+-]?\d+)?)\s*,\s*"
    r"valid\s+loss:\s*([+-]?\d+(?:\.\d+)?(?:[eE][+-]?\d+)?)\s*,\s*accuracy:\s*([+-]?\d+(?:\.\d+)?(?:[eE][+-]?\d+)?)",
    re.IGNORECASE
)

def _norm_key(k: str) -> str:
    """规范化键名：去首尾空白、小写、空格->下划线。绝不合并不同语义的键。"""
    return k.strip().lower().replace(" ", "_")

def parse_log(filepath: str) -> List[Dict[str, float]]:
    """
    解析单个日志文件，按 epoch 汇总所有出现的指标。
    归属规则：位于 'epoch i summary:' 与下一条 'epoch j summary:' 之间的所有 'key : value'
              以及中间出现的块状汇总行，均归到 epoch i。
    返回：List[Dict]，每个元素是一条记录，至少包含：
        - 'epoch': int
      以及该 epoch 出现过的所有指标（如：
        'train_cos_loss', 'train_cro_loss', 'train_tot_loss',
        'valid_cos_loss', 'valid_cro_loss', 'valid_tot_loss',
        'train_loss', 'valid_loss', 'train_accuracy', 'valid_accuracy', ...）
    """
    records: List[Dict[str, float]] = []
    cur_epoch = None
    cur_metrics: Dict[str, float] = {}

    def _flush():
        if cur_epoch is not None:
            rec = {"epoch": cur_epoch}
            rec.update(cur_metrics)
            records.append(rec)

    with open(filepath, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            # 1) 新的 epoch 头
            m_epoch = _RE_EPOCH_HDR.search(line)
            if m_epoch:
                _flush()
                cur_epoch = int(m_epoch.group(1))
                cur_metrics = {}

                # 同行若带有 "key : value"，一并解析
                for k, v in _RE_KV_PAIRS.findall(line):
                    key = _norm_key(k)
                    # 块状的裸 "accuracy" 不在这里出现；这里只记录带前缀的键
                    if key == "accuracy":
                        continue
                    cur_metrics[key] = float(v)
                continue

            # 还未进入任何 epoch，跳过
            if cur_epoch is None:
                continue

            # 2) 每 k 个 epoch 的块状汇总（四个值）
            m_blk = _RE_BLOCK.search(line)
            if m_blk:
                tr_loss, tr_acc, va_loss, va_acc = m_blk.groups()
                cur_metrics["train_loss"] = float(tr_loss)
                cur_metrics["train_accuracy"] = float(tr_acc)
                cur_metrics["valid_loss"] = float(va_loss)
                cur_metrics["valid_accuracy"] = float(va_acc)
                continue

            # 3) 其它分散的 "key : value"
            for k, v in _RE_KV_PAIRS.findall(line):
                key = _norm_key(k)
                # 避免把裸 "accuracy"（无 train/valid 前缀）混入；块状已专门处理
                if key == "accuracy":
                    continue
                cur_metrics[key] = float(v)

    # 文件结束，flush 最后一个 epoch
    _flush()

    # 按 epoch 排序，保证有序
    records.sort(key=lambda r: r["epoch"])
    return records

# --------- 抽取通用 XY 序列（解析阶段与绘图阶段解耦） ---------
def extract_xy(records: List[Dict[str, float]], metric_key: str) -> Tuple[List[int], List[float]]:
    """从某一日志的 epoch 记录中抽取指定指标的 (epochs, values)。缺失的 epoch 会被跳过。"""
    xs, ys = [], []
    for rec in records:
        if metric_key in rec:
            xs.append(rec["epoch"])
            ys.append(rec[metric_key])
    return xs, ys

def parse_many(files: List[str]) -> List[List[Dict[str, float]]]:
    """批量解析：仅做文件->数据结构的转换，不做任何绘图。"""
    return [parse_log(fp) for fp in files]

# --------- 作图（仅接受数据，不再读文件） ---------
def plot_metric_over_epochs(
    parsed_records: List[List[Dict[str, float]]],
    names: List[str],
    metric_key: str,
    title: str = None,
    ylabel: str = None,
    x_lim: Tuple[float, float] = None,
    y_lim: Tuple[float, float] = None,
):
    """
    在一张图上绘制多条实验曲线：metric_key vs epoch。
    参数：
      - parsed_records: 与 names 对应的一组记录列表（即 parse_many 的返回结果）
      - names: 曲线图例名
      - metric_key: 需要绘制的指标名（如 'valid_tot_loss' / 'valid_cos_loss' / 'valid_loss'）
    """
    assert len(parsed_records) == len(names), "parsed_records 与 names 长度不一致。"

    plt.figure(figsize=(8, 5))
    any_plotted = False
    for recs, name in zip(parsed_records, names):
        xs, ys = extract_xy(recs, metric_key)
        if xs:
            plt.plot(xs, ys, label=name)
            any_plotted = True
        else:
            print(f"[WARN] '{name}' 中未找到指标键 '{metric_key}'，已跳过。")

    if not any_plotted:
        print(f"[WARN] 没有任何曲线可画：所有日志都缺少 '{metric_key}'。")
        return

    plt.xlabel("epoch")
    plt.ylabel(ylabel or metric_key)
    plt.title(title or f"{metric_key} vs epoch")
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    if x_lim:
        plt.xlim(x_lim)
    if y_lim:
        plt.ylim(y_lim)
    
    
    plt.show()

# ========== 示例主入口（解析与绘图解耦） ==========
if __name__ == "__main__":
    assert len(LOG_FILES) == len(LOG_NAMES), "LOG_FILES 与 LOG_NAMES 长度不一致。"

    # 仅解析，不作图
    all_records = parse_many(LOG_FILES)  # List[List[Dict]]

    # 需要画哪条指标，调用一次即可；互不影响
    # 例如画 validation 的三种不同定义（各自独立的 key）：
    # plot_metric_over_epochs(all_records, LOG_NAMES, metric_key="valid_tot_loss",
    #                         title="valid_tot_loss vs epoch", ylabel="valid_tot_loss")
    # plot_metric_over_epochs(all_records, LOG_NAMES, metric_key="valid_cos_loss",
    #                         title="valid_cos_loss vs epoch", ylabel="valid_cos_loss")
    plot_metric_over_epochs(all_records, LOG_NAMES, metric_key="valid_loss",
                            title="valid_loss vs epoch", ylabel="valid_loss",
                            y_lim=(1.6, 2.0))

    # 你本次需求：绘制“validation loss 随着 epoch 变化”的曲线
    # 若你用的是块状行里的 "valid loss:"，则 metric_key = "valid_loss"
    # 若你用的是 summary 行里的 "valid_tot_loss:"，则 metric_key = "valid_tot_loss"
    # 下面给出一个默认示例（如需更换，改 metric_key 即可）：
    # plot_metric_over_epochs(all_records, LOG_NAMES, metric_key="valid_loss",
    #                         title="Validation Loss vs Epoch", ylabel="validation loss")
